In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "\n",
    "import tkinter\n",
    "import SimpleGUICS2Pygame.simpleguics2pygame as simplegui\n",
    "\n",
    "# Constants\n",
    "HEIGHT = 400\n",
    "WIDTH = 500\n",
    "NODE_SPACE_ALLOWANCE = 20\n",
    "EDGE_COLOR = \"Yellow\"\n",
    "EDGE_SIZE = 2\n",
    "NODE_LABEL_COLOR = \"White\"\n",
    "NODE_COLOR = \"Red\"\n",
    "NODE_MARK_COLOR = \"Green\"\n",
    "\n",
    "# Global variables\n",
    "start = 0\n",
    "goal = 0\n",
    "placeNodes = True\n",
    "setNodesRelation = False\n",
    "draw_relations = False\n",
    "draw_mark_relations =  False\n",
    "setGoal = False\n",
    "setStart = False\n",
    "displayResult = False\n",
    "lock_nodes = False\n",
    "nodes = []\n",
    "\n",
    "pos1 = [0,0]\n",
    "pos2 = [0,0]\n",
    "pos_lock = False\n",
    "indx = 0\n",
    "letter_label_default = '@'\n",
    "letter_pos = 1\n",
    "current_node_letters_low = []\n",
    "current_node_letters_up = []\n",
    "\n",
    "\n",
    "class Point:\n",
    "    def __init__(self,pos,node_colour,node_mark_colour):\n",
    "        self.pos = pos\n",
    "        self.children = []\n",
    "        self.radius = 5\n",
    "        self.colour = node_colour\n",
    "        self.mark_colour = node_mark_colour\n",
    "        self.index = 0\n",
    "        self.is_mark = False\n",
    "        self.label = '@'\n",
    "\n",
    "    def draw(self,canvas):\n",
    "        if self.is_mark == False:\n",
    "            canvas.draw_circle(self.pos, self.radius, 6, self.colour)\n",
    "        else:\n",
    "            canvas.draw_circle(self.pos, self.radius, 6, self.mark_colour)\n",
    "\n",
    "\n",
    "\n",
    "def mouseclick(pos):\n",
    "    global pos1, pos2, pos_lock, indx, draw_relations, draw_mark_relations, nodes, indx_mark_color\n",
    "    global letter_label_default, letter_pos\n",
    "    \n",
    "    # Creates new instance of point(node) if the last position of\n",
    "    # the mouseclick is not on  top of a previous node\n",
    "    allow_place_node = True\n",
    "    \n",
    "    if placeNodes == True:\n",
    "        if nodes: # Checks if the nodes are not empty\n",
    "            for p, location in enumerate(nodes):\n",
    "                if ((pos[0] >= (nodes[p].pos[0]-NODE_SPACE_ALLOWANCE) and pos[0] <= (nodes[p].pos[0]+NODE_SPACE_ALLOWANCE)) and\n",
    "                    (pos[1] >= (nodes[p].pos[1]-NODE_SPACE_ALLOWANCE) and pos[1] <= (nodes[p].pos[1]+NODE_SPACE_ALLOWANCE))):\n",
    "                    print( \"Warning: Cannot create node on top of another node!\")\n",
    "                    allow_place_node = False\n",
    "                    break\n",
    "            # Creates new instance of Point class if no nodes detected in \n",
    "            # the vicinity of mouseclick position\n",
    "            if allow_place_node == True:\n",
    "                nodes.append(Point(pos,NODE_COLOR,NODE_MARK_COLOR))\n",
    "                nodes[-1].label = chr(ord(letter_label_default) + letter_pos)\n",
    "                letter_pos += 1\n",
    "        # Else creates a node for first time \n",
    "        else:  \n",
    "            nodes.append(Point(pos,NODE_COLOR,NODE_MARK_COLOR))\n",
    "            nodes[-1].label = chr(ord(letter_label_default) + letter_pos)\n",
    "            letter_pos += 1\n",
    "    \n",
    "    # Sets up the edges or links\n",
    "    if setNodesRelation == True:\n",
    "        \n",
    "        # If mouseclick pos is on top of a current node mark that node\n",
    "        for i, position in enumerate(nodes):\n",
    "            if ((pos[0] >= (nodes[i].pos[0]-NODE_SPACE_ALLOWANCE) and pos[0] <= (nodes[i].pos[0]+NODE_SPACE_ALLOWANCE)) and\n",
    "                (pos[1] >= (nodes[i].pos[1]-NODE_SPACE_ALLOWANCE) and pos[1] <= (nodes[i].pos[1]+NODE_SPACE_ALLOWANCE))):\n",
    "                if pos_lock == False:\n",
    "                    pos1[0] = pos[0]\n",
    "                    pos1[1] = pos[1]\n",
    "                    \n",
    "                    indx = i\n",
    "                    indx_mark_color = i\n",
    "                    pos_lock = True\n",
    "                    draw_mark_relations = True\n",
    "                    break\n",
    "\n",
    "                else:\n",
    "                    # If it is the second node that is not the same of \n",
    "                    # the first marked node, then creates a new relation/edge\n",
    "                    if i != indx:\n",
    "                        pos2[0] = pos[0]\n",
    "                        pos2[1] = pos[1]\n",
    "                        nodes[indx].children.append(i)\n",
    "                        nodes[i].children.append(indx)\n",
    "                        \n",
    "                        pos_lock = False\n",
    "                        draw_relations = True\n",
    "                        draw_mark_relations = False\n",
    "                        break\n",
    "                    else:\n",
    "                        print (\"Warning: Recursion or self loop is not allowed.\")\n",
    "\n",
    "\n",
    "def button_refresh_new_relation():\n",
    "    global pos_lock, pos1, pos2, nodes, draw_relations, draw_mark_relations\n",
    "    \n",
    "    if lock_nodes == False and setNodesRelation == True:\n",
    "        pos_lock = False\n",
    "        draw_mark_relations = False\n",
    "        draw_relations = False\n",
    "        pos1[0] = 0\n",
    "        pos1[1] = 0\n",
    "        pos2[0] = 0\n",
    "        pos2[1] = 0\n",
    "\n",
    "        # This empties the list of children attribute of Point class\n",
    "        for i, child in enumerate(nodes):\n",
    "            del nodes[i].children[:]\n",
    "    else:\n",
    "        print( \"Warning: This action is not allowed.\")\n",
    "\n",
    "\n",
    "def button_lock_nodes():\n",
    "    global placeNodes, setNodesRelation, current_node_letters_up, nodes, current_node_letters_low\n",
    "    \n",
    "    # Can only lock nodes if the set-up is right\n",
    "    # Prevents locking nodes later in the program\n",
    "    if placeNodes == True and setNodesRelation == False and setStart == False and setGoal == False:\n",
    "        placeNodes = False\n",
    "        setNodesRelation = True\n",
    "        \n",
    "        # Fills two new lists of all node labels(letters) \n",
    "        # for later use in input start and goal\n",
    "        if nodes:\n",
    "            for n, obj in enumerate(nodes):\n",
    "                current_node_letters_up.append(nodes[n].label)\n",
    "\n",
    "            for let in current_node_letters_up:\n",
    "                current_node_letters_low.append(let.lower())\n",
    "\n",
    "        print( \"The nodes are now locked in!\")\n",
    "    else:\n",
    "        print (\"Warning: This action is not allowed.\")\n",
    "\n",
    "def button_lock_graph():\n",
    "    global placeNodes, setNodesRelation, nodes, lock_nodes\n",
    "    \n",
    "    if setNodesRelation is True:\n",
    "        placeNodes = False\n",
    "        setNodesRelation = False\n",
    "        lock_nodes = True\n",
    "\n",
    "        # Sets the index of nodes list and apply it to each index attribute of Point class\n",
    "        # for index/element reference only, for later use in BFS and DFS functions\n",
    "        for d, dot in enumerate(nodes):\n",
    "            nodes[d].index = d\n",
    "            print( \"node\"+str(d+1)+\":\", nodes[d].label)\n",
    "            \n",
    "            # This is important\n",
    "            # This sorts the elements of children attribute list in ascending order\n",
    "            nodes[d].children.sort()\n",
    "\n",
    "        print (\"Graph is now set!\")\n",
    "    else:\n",
    "        print( \"Warning: This action is not allowed.\")\n",
    "\n",
    "\n",
    "def input_start_handler(start_string):\n",
    "    \n",
    "    global start, nodes, setStart\n",
    "    \n",
    "    setStart = False\n",
    "    if start_string.isdigit():\n",
    "        # Allows number as input for starting node\n",
    "        # 1 for A, 2 for B and so on and so forth\n",
    "        temp_start = int(start_string) - 1\n",
    "        for element, num in enumerate(nodes):\n",
    "            if temp_start == element:\n",
    "                \n",
    "                # Minus one because node label starts at 1 not zero(index)\n",
    "                start = temp_start\n",
    "                print( \"Start:\", chr(start+65))\n",
    "                setStart = True\n",
    "                break \n",
    "        if setStart == False:  \n",
    "            print( \"Warning: This number is outside of the nodes!\")\n",
    "    else:\n",
    "        # Allows letter as input for starting node\n",
    "        if start_string in current_node_letters_up:\n",
    "            start = ord(start_string) - 65\n",
    "            setStart = True\n",
    "            print (\"Start:\", chr(start+65))\n",
    "        else:\n",
    "            if start_string in current_node_letters_low:\n",
    "                start = ord(start_string) - 97\n",
    "                setStart = True\n",
    "                print( \"Start:\", chr(start+65))\n",
    "            else:\n",
    "                print( \"Warning: Unknown input. Enter a number or the node letter.\")\n",
    "\n",
    "\n",
    "def input_goal_handler(goal_string):\n",
    "    global goal, nodes, setGoal\n",
    "    \n",
    "    setGoal = False\n",
    "    if goal_string.isdigit():\n",
    "        \n",
    "        # Allows number as input for goal node\n",
    "        # 1 for A, 2 for B and so on and so forth\n",
    "        temp_goal = int(goal_string) - 1\n",
    "        for element, num in enumerate(nodes):\n",
    "            if temp_goal == element:\n",
    "                #minus one because node label starts at 1 not zero(index)\n",
    "                goal = temp_goal\n",
    "                print( \"Goal:\", chr(goal+65))\n",
    "                setGoal = True\n",
    "                break\n",
    "        if setGoal == False:\n",
    "            print( \"Warning: This number is outside of the nodes!\")\n",
    "    else:\n",
    "        # Allows letter as input for goal node\n",
    "        if goal_string in current_node_letters_up:\n",
    "            goal = ord(goal_string) - 65\n",
    "            setGoal = True\n",
    "            print( \"Goal:\", chr(goal+65))\n",
    "        else:\n",
    "            if goal_string in current_node_letters_low:\n",
    "                goal = ord(goal_string) - 97\n",
    "                setGoal = True\n",
    "                print (\"Goal:\", chr(goal+65))\n",
    "            else:\n",
    "                print( \"Warning: Unknown input. Enter a number or the node letter.\")\n",
    "\n",
    "\n",
    "def button_breadth_first_search():\n",
    "    global nodes, displayResult, result_string, queue_string, pointer_string\n",
    "    displayResult = False\n",
    "    pointer_string = \"\"\n",
    "\n",
    "    # Resets all nodes markings (color)\n",
    "    for d, marking_obj in enumerate(nodes):\n",
    "        nodes[d].is_mark = False\n",
    "\n",
    "    in_queue_result = False\n",
    "\n",
    "    if placeNodes == False and setNodesRelation == False and setStart == True and setGoal == True:\n",
    "        print( \" \")\n",
    "        print( \"BFS starts here:\")\n",
    "        \n",
    "        # Checks queue if defined, \n",
    "        # if it is, then go to else and empty the list; otherwise create a new list\n",
    "        try:\n",
    "            queue\n",
    "        except:\n",
    "            queue = []\n",
    "        else:\n",
    "            del queue[:]\n",
    "\n",
    "        queue.append(nodes[start])\n",
    "        queue[0].is_mark = True\n",
    "        \n",
    "        try:\n",
    "            result\n",
    "        except:\n",
    "            result = []\n",
    "        else:\n",
    "            del result[:]\n",
    "        \n",
    "        while queue:\n",
    "            pointer = queue[0]\n",
    "            queue.pop(0)\n",
    "            \n",
    "            pointer.is_mark = True\n",
    "            print( \" \")\n",
    "            print (\"Pointer:\", pointer.label)\n",
    "            \n",
    "            if pointer.index == goal:\n",
    "                pointer_string =  \"Pointer: \" + pointer.label\n",
    "                result_string = \"Result: \"\n",
    "                queue_string = \"Queue: \"\n",
    "                \n",
    "                for obj in result:\n",
    "                    result_string += str(obj.label)\n",
    "                    result_string += \" \"\n",
    "                for objt in queue:\n",
    "                    queue_string += str(objt.label)\n",
    "                    queue_string += \" \"\n",
    "                \n",
    "                displayResult = True\n",
    "                print (\"SUCCESS!\")\n",
    "                break\n",
    "            else:\n",
    "                result.append(pointer)\n",
    "\n",
    "                for neighbor in pointer.children:\n",
    "                    in_queue_result = False\n",
    "                    for i in queue:\n",
    "                        #print \"neighbor:\", neighbor+1, \"queue:\", i.index+1\n",
    "                        if neighbor == i.index:\n",
    "                            in_queue_result = True\n",
    "                            \n",
    "                    for j in result:\n",
    "                        #print \"neighbor:\", neighbor+1, \"result:\", j.index+1\n",
    "                        if neighbor == j.index:\n",
    "                            in_queue_result = True\n",
    "\n",
    "                    if in_queue_result == False:\n",
    "                        for objct in nodes:\n",
    "                            if objct.index == neighbor:\n",
    "                                queue.append(nodes[objct.index])\n",
    "            result_string = \"Result: \"\n",
    "            queue_string = \"Queue: \"\n",
    "            for obj in result:\n",
    "                result_string += str(obj.label)\n",
    "                result_string += \" \"\n",
    "            print (result_string)\n",
    "\n",
    "            for objt in queue:\n",
    "                queue_string += str(objt.label)\n",
    "                queue_string += \" \"\n",
    "            print( queue_string)\n",
    "                    \n",
    "                    \n",
    "def button_depth_first_search():\n",
    "    \n",
    "    global nodes, displayResult, result_string, queue_string, pointer_string\n",
    "    displayResult = False\n",
    "    pointer_string = \"\"\n",
    "\n",
    "# Resets all nodes markings (color)\n",
    "    for d, marking_obj in enumerate(nodes):\n",
    "        nodes[d].is_mark = False\n",
    "\n",
    "    in_queue_result = False\n",
    "\n",
    "    if placeNodes == False and setNodesRelation == False and setStart == True and setGoal == True:\n",
    "        print( \" \")\n",
    "        print (\"DFS starts here:\")\n",
    "        \n",
    "        # Checks queue if defined, \n",
    "        # if it is, then go to else and empty the list; otherwise create new list\n",
    "        try:\n",
    "            queue\n",
    "        except:\n",
    "            queue = []\n",
    "        else:\n",
    "            del queue[:]\n",
    "        \n",
    "        #print queue\n",
    "        queue.append(nodes[start])\n",
    "        queue[0].is_mark = True\n",
    "        #print \"queue:\", queue\n",
    "        try:\n",
    "            result\n",
    "        except:\n",
    "            result = []\n",
    "        else:\n",
    "            del result[:]\n",
    "\n",
    "\n",
    "        try:\n",
    "            temp_list\n",
    "        except:\n",
    "            temp_list = []\n",
    "        else:\n",
    "            del temp_list[:]\n",
    "         \n",
    "\n",
    "        while queue:\n",
    "            pointer = queue[0]\n",
    "            queue.pop(0)\n",
    "            #print \"pointer is\", pointer\n",
    "            pointer.is_mark = True\n",
    "            print (\" \")\n",
    "            print( \"Pointer:\", pointer.label)\n",
    "            \n",
    "            \n",
    "            if pointer.index == goal:\n",
    "                pointer_string =  \"Pointer: \" + pointer.label\n",
    "                result_string = \"Result: \"\n",
    "                queue_string = \"Queue: \"\n",
    "                \n",
    "                for obj in result:\n",
    "                    result_string += str(obj.label)\n",
    "                    result_string += \" \"\n",
    "                for objt in queue:\n",
    "                    queue_string += str(objt.label)\n",
    "                    queue_string += \" \"\n",
    "                \n",
    "                displayResult = True\n",
    "                print( \"SUCCESS!\")\n",
    "                break\n",
    "            else:\n",
    "                result.append(pointer)\n",
    "                del temp_list[:]\n",
    "                \n",
    "                for neighbor in pointer.children:\n",
    "                    in_queue_result = False\n",
    "\n",
    "                    for i in queue:\n",
    "                        #print \"neighbor:\", neighbor+1, \"queue:\", i.index+1\n",
    "                        if neighbor == i.index:\n",
    "                            in_queue_result = True\n",
    "                            \n",
    "                    for j in result:\n",
    "                        #print \"neighbor:\", neighbor+1, \"result:\", j.index+1\n",
    "                        if neighbor == j.index:\n",
    "                            in_queue_result = True\n",
    "\n",
    "                    if in_queue_result == False:\n",
    "                        for obj in nodes:\n",
    "                            if obj.index == neighbor:\n",
    "                                temp_list.append(nodes[obj.index])\n",
    "                \n",
    "                if temp_list:      \n",
    "                    queue[0:0] = temp_list\n",
    "\n",
    "            result_string = \"Result: \"\n",
    "            queue_string = \"Queue: \"\n",
    "            for obj in result:\n",
    "                result_string += str(obj.label)\n",
    "                result_string += \" \"\n",
    "            print( result_string)\n",
    "\n",
    "            for objt in queue:\n",
    "                queue_string += str(objt.label)\n",
    "                queue_string += \" \"\n",
    "            print( queue_string  )\n",
    "\n",
    "\n",
    "def draw_handler(canvas):\n",
    "    global result_string, queue_string, pointer_string\n",
    "    global placeNodes, setNodesRelation, setStart, setGoal, pos1\n",
    "    \n",
    "    canvas.draw_image(image, (500, 500), (1521, 1818), (50, 50), (100, 100))\n",
    "    # Draws nodes\n",
    "    if draw_mark_relations == True and setNodesRelation == True:\n",
    "        canvas.draw_circle(nodes[indx_mark_color].pos, 15, 3, \"Yellow\", \"Black\")\n",
    "\n",
    "    if nodes: \n",
    "        for i, vertex in enumerate(nodes):\n",
    "            nodes[i].draw(canvas)\n",
    "            canvas.draw_text(nodes[i].label, (nodes[i].pos[0]-30, nodes[i].pos[1]), 20, NODE_LABEL_COLOR)\n",
    "\n",
    "    # Draws edges\n",
    "    if draw_relations == True:\n",
    "        for n, point in enumerate(nodes):\n",
    "            if nodes[n].children: \n",
    "                for child in nodes[n].children:\n",
    "                    canvas.draw_line(nodes[n].pos, nodes[child].pos, EDGE_SIZE, EDGE_COLOR)\n",
    "\n",
    "    # Display results\n",
    "    if displayResult == True:\n",
    "        canvas.draw_text(pointer_string, (30, 345), 15, NODE_LABEL_COLOR)\n",
    "        canvas.draw_text(result_string, (30, 370), 15, NODE_LABEL_COLOR)\n",
    "        canvas.draw_text(queue_string, (30, 395), 15, NODE_LABEL_COLOR)\n",
    "\n",
    "\n",
    "# Creates the frame window\n",
    "frame = simplegui.create_frame(\"Graph Node Search with BFS and DFS\",WIDTH,HEIGHT)\n",
    "\n",
    "frame.set_mouseclick_handler(mouseclick)\n",
    "frame.set_draw_handler(draw_handler)\n",
    "\n",
    "# Button, input and label controls for the frame window\n",
    "image = simplegui.load_image('http://www.shafaff.com/uploads/news_major_image/2015/02/05/1423132571/3f1e203d-82e8-4e3c-a14c-5a98a9a3b151.png')\n",
    "button1 = frame.add_button('Lock in the nodes', button_lock_nodes)\n",
    "label1 = frame.add_label(' ')\n",
    "\n",
    "button2 = frame.add_button('Lock in the graph', button_lock_graph)\n",
    "label2 = frame.add_label(' ')\n",
    "\n",
    "button3 = frame.add_button('Reset edge drawing', button_refresh_new_relation)\n",
    "label3 = frame.add_label(' ')\n",
    "\n",
    "input_start = frame.add_input('Set start', input_start_handler, 50)\n",
    "label4 = frame.add_label(' ')\n",
    "\n",
    "input_goal = frame.add_input('Set goal', input_goal_handler, 50)\n",
    "label5 = frame.add_label(' ')\n",
    "\n",
    "button4 = frame.add_button('Find BFS', button_breadth_first_search)\n",
    "button5 = frame.add_button('Find DFS', button_depth_first_search)\n",
    "\n",
    "label6 = frame.add_label(' ')\n",
    "label7 = frame.add_label('tasneem said, sec: 1 ')\n",
    "label8 = frame.add_label('Dr: Ahmed Mohamed ')\n",
    "\n",
    "\n",
    "# Program starts here\n",
    "frame.start()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
